In [1]:
import ipywidgets as ipw
import subprocess

In [2]:
from utils.control import *

In [3]:
msg,updates_needed,config = check_for_updates('s1267')
#tutto = check_for_updates()

In [4]:
 msg

"✅ The .ssh/config seems to be OK.<br>✅ Active AiiDA computers: ✅daint.alps_s1267<br>✅localhost<br>✅⬜ Not active AiiDA computers: ✅⬜202503070709_daint.alps_s1267<br>✅⬜Used_till_202502100831_daint.alps<br>✅⬜202503070700_daint.alps_s1267<br>✅⬜202503061611_daint.alps_s1267<br>✅⬜Used_till_202502100724_daint.alps<br>✅⬜Used_till_202502120713_daint.alps<br>✅⬜202503061643_daint.alps_s1267<br>✅⬜Used_till_202502120922_daint.alps<br>✅⬜Used_till_202502091226_daint.alps<br>✅⬜Used_till_202502120836_daint.alps<br>✅⬜Used_till_202502100738_daint.alps<br>✅⬜daint.alps<br>✅⬜202503070725_daint.alps_s1267<br>✅⬜202503070715_daint.alps_s1267<br>✅ Active AiiDA codes:<br> ✅ stm@daint.alps  PK: 15191<br>✅ pw-7.4:v2@daint.alps  PK: 15185<br>✅ projwfc-7.4:v2@daint.alps_s1267@daint.alps  PK: 15213<br>✅ python-py39@daint.alps_s1267@daint.alps  PK: 15200<br>✅ projwfc-7.4:v2@daint.alps_s1267@daint.alps  PK: 15246<br>✅ pw-7.4:v2@daint.alps_s1267@daint.alps  PK: 15201<br>✅ pw2gw-7.2@localhost  PK: 1828<br>✅ open_grid-7.

In [5]:
config

{'python_version': '3.9.13',
 'new_host_label': 'daint.alps',
 'remotehost': 'daint.alps.cscs.ch',
 'proxy': 'ela.cscs.ch',
 'grants': {'daint.alps': ['s1267', 's1276', 's1295'], 'tigu': ['free']},
 'computers': {'daint.alps': {'setup': {'description': 'daint.alps at CSCS',
    'hostname': 'daint.alps.cscs.ch',
    'label': 'daint.alps',
    'mpiprocs_per_machine': 256,
    'mpirun_command': 'srun --cpu-bind=socket /users/cscsusername/bin/mps-wrapper.sh',
    'prepend_text': '#SBATCH --ntasks-per-core=1\n#SBATCH --hint=nomultithread\n#SBATCH --no-requeue\n#SBATCH --account=cscsaccount\n',
    'scheduler': 'core.slurm',
    'shebang': '#!/bin/bash -l',
    'transport': 'core.ssh',
    'use_double_quotes': False,
    'work_dir': '/capstor/scratch/cscs/{username}/aiida/',
    'default_memory_per_machine': 268435456},
   'config': {'allow_agent': True,
    'compress': True,
    'gss_auth': False,
    'gss_deleg_creds': False,
    'gss_host': 'daint.alps.cscs.ch',
    'gss_kex': False,
    

In [6]:
updates_needed

{'computers': {'daint.alps_s1267': {'hide': True,
   'rename': True,
   'install': True}},
 'codes': {'stm-1.5.0@daint.alps_s1267': {'rename': False, 'install': True},
  'python-py39@daint.alps_s1267': {'rename': False, 'install': True},
  'pw-7.4:v2@daint.alps_s1267': {'rename': False, 'install': True},
  'pp-7.4:v2@daint.alps_s1267': {'rename': False, 'install': True},
  'projwfc-7.4:v2@daint.alps_s1267': {'rename': False, 'install': True},
  'dos-7.4:v2@daint.alps_s1267': {'rename': False, 'install': True},
  'phonopy@daint.alps_s1267': {'rename': False, 'install': True},
  'overlap-1.5.0@daint.alps_s1267': {'rename': False, 'install': True},
  'critic2@daint.alps_s1267': {'rename': False, 'install': True},
  'cp2k-2024.3:v2@daint.alps_s1267': {'rename': False, 'install': True}}}

In [7]:
subprocess.run(configure_command, capture_output=True, text=True, check=True)

NameError: name 'configure_command' is not defined

In [ ]:
a={}

In [ ]:
a.setdefault('b',{})['c']=12

In [ ]:
a.setdefault('b',{})['d']=13

In [ ]:
a

In [ ]:
a=[""]+['asa','besa']

In [ ]:
a

In [ ]:
'-as-'.join(['1','2','3'])